In [1]:
DEBUG = False # set this flag to True to use a small subset of data for testing
     

In [7]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Using cached fastai-2.7.10-py3-none-any.whl (240 kB)
  Using cached torchvision-0.14.1-cp310-cp310-manylinux1_x86_64.whl (24.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 2.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 846.9 kB/s eta 0:00:00m eta 0:00:010:00:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 890.5 kB/s eta 0:00:00m eta 0:00:010:00:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 2.2 MB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached fastdownload-0.0.7-py3-none-any.whl (12 kB)
  Using cached spacy-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Using cached fastcore-1.5.27-py3-none-any.whl (67 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 4.0 MB/s eta 0:00:000:00:010:00:01
     ━━━

In [2]:
2+2

4

In [3]:
from fastai.vision.all import *
import params

import wandb
     

We have defined some global configuration parameters in the params.py file. ENTITY should correspond to your W&B Team name if you work in a team, replace it with None if you work individually.

In the section below, we will use untar_data function from fastai to download and unzip our datasets.

In [6]:
URL = 'https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip'

In [8]:

path = Path(untar_data(URL, force_download=True))

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [9]:
path.ls()

NameError: name 'path' is not defined

Here we define several functions to help us process the data and upload it as a Table to W&B.

In [5]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "None", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )
    
    return table

We will start a new W&B run and put everything into a raw Artifact.

In [6]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="upload")
raw_data_at = wandb.Artifact(params.RAW_DATA_AT, type="raw_data")
     

wandb: Currently logged in as: pashikantipranith7867. Use `wandb login --relogin` to force relogin


In [7]:
raw_data_at.add_file(path/'LICENSE.txt', name='LICENSE.txt')

ArtifactManifestEntry(path='LICENSE.txt', digest='X+6ZFkDOlnKesJCNt20yRg==', ref=None, birth_artifact_id=None, size=1594, extra={}, local_path='/home/pranith/.local/share/wandb/artifacts/staging/tmpqrnv1hlp')

Let's add the images and label masks.

In [8]:

raw_data_at.add_dir(path/'images', name='images')
raw_data_at.add_dir(path/'labels', name='labels')

wandb: Adding directory to artifact (/home/pranith/.fastai/data/bdd_simple_1k/images)... Done. 7.9s
wandb: Adding directory to artifact (/home/pranith/.fastai/data/bdd_simple_1k/labels)... Done. 0.6s


Let's get the file names of images in our dataset and use the function we defined above to create a W&B Table.

In [11]:

image_files = get_image_files(path/"images", recurse=False)

# sample a subset if DEBUG
if DEBUG: image_files = image_files[:10]

In [12]:
table = _create_table(image_files, params.BDD_CLASSES)

Finally, we will add the Table to our Artifact, log it to W&B and finish our run.

In [13]:

raw_data_at.add(table, "eda_table")

ArtifactManifestEntry(path='eda_table.table.json', digest='7E5ta3A3iov5jcGOb0t3JQ==', ref=None, birth_artifact_id=None, size=588824, extra={}, local_path='/home/pranith/.local/share/wandb/artifacts/staging/tmp14iwno2u')

In [14]:

run.log_artifact(raw_data_at)
run.finish()

In [ ]:
2+2